In [134]:
import mysql.connector
import pandas as pd
import numpy as np
#import seaborn as sns
#import requests
#import json
import datetime as dt
#import time
import matplotlib.pyplot as plt

In [135]:
# connect to mysql
cnx = mysql.connector.connect(user='nba_user', password='letmein',
                              host='127.0.0.1',
                              database='nba_data')
cursor = cnx.cursor()

In [136]:
# call table from MySQL
sql = "SELECT * FROM nba;"

cursor.execute(sql)
rows = cursor.fetchall();

In [137]:
column_names = ['game_time', 'game_date', 'arena', 'home_team', 'home_wins', 
                'home_losses', 'away_team', 'away_wins', 'away_losses', 'attendance']

In [138]:
# create dataframe
features = pd.DataFrame(rows, columns=column_names)

In [139]:
features.to_csv('NBA_data.csv')

In [140]:
features.drop(columns='game_time', inplace=True)

In [141]:
features.head(5)

,game_date,arena,home_team,home_wins,home_losses,away_team,away_wins,away_losses,attendance
0,2017-10-17,Quicken Loans Arena,Cleveland Cavaliers,1,0,Boston Celtics,0,1,20562
1,2017-10-17,Oracle Arena,Golden State Warriors,0,1,Houston Rockets,1,0,19596
2,2017-10-18,TD Garden,Boston Celtics,0,2,Milwaukee Bucks,1,0,18624
3,2017-10-18,American Airlines Center,Dallas Mavericks,0,1,Atlanta Hawks,1,0,19709
4,2017-10-18,Little Caesars Arena,Detroit Pistons,1,0,Charlotte Hornets,0,1,20491


In [142]:
# limit to categorical data using df.select_dtypes()
#X = features.select_dtypes(include=[object])
#X.head(10)

In [143]:
features['game_date'] = pd.to_datetime(features['game_date'])

In [144]:
#features['weekday'] = features['game_date'].dt.strftime('%A')

In [145]:
#features['game_date'] = features['game_date'].map(dt.datetime.toordinal)

In [146]:
features.game_date.dtype

dtype('<M8[ns]')

In [147]:
# Create new columns for day, month, year, and day of the week
features['day'] = features['game_date'].dt.day
features['month'] = features['game_date'].dt.month
features['year'] = features['game_date'].dt.year
features['weekday'] = features['game_date'].dt.strftime('%A')
features.drop(columns='game_date', inplace=True)

In [148]:
# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(features)
# Display the first 5 rows
features.head(5)

,home_wins,home_losses,away_wins,away_losses,attendance,day,month,year,arena_AT&T Center,arena_AccorHotels Arena,...,away_team_Toronto Raptors,away_team_Utah Jazz,away_team_Washington Wizards,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,1,0,0,1,20562,17,10,2017,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,1,1,0,19596,17,10,2017,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,2,1,0,18624,18,10,2017,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,1,0,19709,18,10,2017,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,1,20491,18,10,2017,0,0,...,0,0,0,0,0,0,0,0,0,1


In [150]:
# Labels are the values we want to predict
labels = np.array(features['attendance'])

# Remove the labels from the features
# axis 1 refers to the columns
features = features.drop('attendance', axis = 1)
feature = features

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

In [153]:
#Get a random sample for prediction later
random_row = feature.sample(n=1)

# Print the sample
print(random_row)

      home_wins  home_losses  away_wins  away_losses  day  month  year  \
3319         16           37         18           36   12      2  2020   

      arena_AT&T Center  arena_AccorHotels Arena  arena_Air Canada Centre  \
3319                  0                        0                        0   

      ...  away_team_Toronto Raptors  away_team_Utah Jazz  \
3319  ...                          0                    0   

      away_team_Washington Wizards  weekday_Friday  weekday_Monday  \
3319                             0               0               0   

      weekday_Saturday  weekday_Sunday  weekday_Thursday  weekday_Tuesday  \
3319                 0               0                 0                0   

      weekday_Wednesday  
3319                  1  

[1 rows x 120 columns]


In [121]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 69)

In [109]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (4386, 120)
Training Labels Shape: (4386,)
Testing Features Shape: (1463, 120)
Testing Labels Shape: (1463,)


In [110]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Import statistic calculations
from sklearn.metrics import mean_squared_error, r2_score

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 69)

# Train the model on training data
rf.fit(train_features, train_labels);

In [111]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 808.04


In [28]:
from sklearn.metrics import mean_absolute_percentage_error

#Calculate MAPE
mape = mean_absolute_percentage_error(test_labels, predictions)
mape = 100 * mape
print('MAPE:',round(mape,2),'%.')

MAPE: 4.98 %.


In [29]:
# Calculate mean absolute percentage error (MAPE)
# mape = 100 * (errors / test_labels)

# Calculate and display accuracy
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 95.02 %.


In [30]:
# Use the regression's predict method on the test data
y_pred = rf.predict(test_features)
msq = mean_squared_error(test_labels, predictions)
r2 = r2_score(test_labels, predictions)
print("Mean Squared Error:", round(msq,4))
print("r2:", round(r2,4))

Mean Squared Error: 1458290.6152
r2: 0.6611


In [154]:
# Run an example through the random forest
prediction = rf.predict(random_row)
print(prediction)

[16948.01]


/Users/elijah/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [209]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
# [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [210]:
#conda install pydot-ng

In [211]:
#conda install graphviz

In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

# Pull out one tree from the forest
tree = rf.estimators_[5]

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
# graph.write_png('tree.png')

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)

# Extract the small tree
tree_small = rf_small.estimators_[5]

# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
#graph.write_png('small_tree.png');

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
# [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [127]:
#import packages for linear regression
from sklearn.linear_model import LinearRegression

In [128]:
# Create a linear regression model
model = LinearRegression()

# Train the model using the training data
model.fit(train_features, train_labels)

LinearRegression()

In [129]:
# Use the regression's predict method on the test data
y_pred = model.predict(test_features)
msq = mean_squared_error(test_labels, y_pred)
r2 = r2_score(test_labels, y_pred)
print("Mean Squared Error:", round(msq,4))
print("r2:", round(r2,4))

Mean Squared Error: 1718082.8758
r2: 0.6008


In [130]:
# Calculate the absolute errors
errors_reg = abs(y_pred - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors_reg), 2))

Mean Absolute Error: 968.32


In [131]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(test_labels, y_pred)
print('Mean Absolute Error:', round(mae, 2))

Mean Absolute Error: 968.32


In [132]:
from sklearn.metrics import mean_absolute_percentage_error

#Calculate MAPE
mape_reg = mean_absolute_percentage_error(test_labels, y_pred)
mape_reg = 100 * mape_reg
print('MAPE:',round(mape_reg,2),'%.')

MAPE: 5.93 %.


In [232]:
#Calculate Accuracy
acc_reg = 100 - mape_reg
print('Accuracy:', round(acc_reg, 2), '%.')

Accuracy: 94.07 %.


In [155]:
# Run an example through the linear regression
prediction = model.predict(random_row)
print(prediction)

[14970.37498427]


/Users/elijah/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
